In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python 

In [1]:
import tensorflow as tf
import os

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
tf.config.list_physical_devices('GPU')

[]

# Removing unclear images

In [4]:
import cv2
import imghdr

In [10]:
data_dir = 'augmented' 

In [11]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [12]:
os.listdir(data_dir)

['cats', 'dogs']

In [13]:
for image_class in os.listdir(data_dir): 
    print(image_class)

cats
dogs


In [15]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# Load Data

In [16]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

Found 19928 files belonging to 2 classes.


In [17]:
data

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

# Scaling Data

In [18]:
#Returns value between 0 and 1
data = data.map(lambda x,y: (x/255, y))

In [19]:
catgry = os.listdir(data_dir)

# Split Data

In [20]:
len(data)

623

In [21]:
#split into train and test

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)+1

In [22]:
print(train_size)
print(val_size)
print(test_size)

436
124
63


In [23]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# Deep Learning Model using CNN

In [24]:
train

<TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [26]:
model = Sequential()

In [27]:
#Add convolution and max-pooling layer
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [28]:
model.compile(optimizer = 'adam',loss = tf.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

# Training

In [30]:
logdir='logs'

In [31]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [32]:
hist = model.fit(train, epochs=1, validation_data=val, callbacks=[tensorboard_callback])

436/436 [==============================] - 342s 782ms/step - loss: 0.6549 - accuracy: 0.6182 - val_loss: 0.6026 - val_accuracy: 0.6948


# Evaluation

In [31]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [32]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [33]:
import numpy as np

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)

    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

1/1 [==============================] - 0s 223ms/step


In [34]:
print(f"precision : {pre.result()}")
print(f"Recall : {re.result()}")
print(f"accuracy : {acc.result()}")

precision : 0.6416465044021606
Recall : 0.5364372730255127
accuracy : 0.6318359375


# Save the Model

In [156]:
model.save(os.path.join('models','imageclassifier.h5'))